In [2]:
import pandas as pd
import requests
import ast
import numpy as np

More info on requests library can be found [here](http://docs.python-requests.org/en/master/user/quickstart/)

# Simple example

In [61]:
# set endpoint URL
url = 'https://data.austintexas.gov/resource/x9yh-78fz.json'
# query params
payload = {'fiscal_year_issued': '2018'}
           'contractor_zip': '78749',
           '$select': 'condominium, total_job_valuation,\
               longitude, latitude, permittype, total_new_add_sqft, \
               total_existing_bldg_sqft'}
# API token
headers = {'X-App-Token': 'HLR3sDnaPUVVakzmZzKPpFMkJ'} 

In [62]:
res = requests.get(url, params=payload, headers=headers)

In [63]:
print(res.status_code)
res.status_code == requests.codes.ok

200


True

In [64]:
# let's take a look at the header information
ast.literal_eval(res.headers['X-SODA2-Fields'])

['applicant_address1',
 'applicant_address2',
 'applicant_city',
 'applicant_full_name',
 'applicant_org',
 'applicant_phone',
 'applicant_phone_type',
 'applicantzip',
 'applieddate',
 'building_valuation',
 'building_valuation_remodel',
 'calendar_year_issued',
 'completed_date',
 'condominium',
 'contractor_address1',
 'contractor_address2',
 'contractor_city',
 'contractor_company_name',
 'contractor_full_name',
 'contractor_phone',
 'contractor_phone_type',
 'contractor_trade',
 'contractor_zip',
 'council_district',
 'day_issued',
 'description',
 'electrical_valuation',
 'electrical_valuation_remodel',
 'expiresdate',
 'fiscal_year_issued',
 'housing_units',
 'issue_date',
 'issue_method',
 'issued_in_last_30_days',
 'jurisdiction',
 'latitude',
 'legal_description',
 'link',
 'link_description',
 'location',
 'location_address',
 'location_city',
 'location_state',
 'location_zip',
 'longitude',
 'masterpermitnum',
 'mechanical_valuation',
 'mechanical_valuation_remodel',
 'med

In [7]:
def get_unique_keys(list_of_dicts):
    '''
    return unique keys from within a list of dicts
    '''
    json_keys = []
    for row in list_of_dicts:
        for k in row.keys():
            json_keys.append(k)
    return set(json_keys)

In [8]:
# create a template dictionary
# consisting of all unique keys
# within input dataset
df_dict = {}
for key in get_unique_keys(res.json()):
    df_dict[key] = []
    
# fill in the dictionary with
# values from the API call
# making sure to input nulls
# where needed
for row in res.json():
    for k in list(df_dict.keys()):
        try:
            df_dict[k].append(row[k])
            continue
        except KeyError:
            df_dict[k].append(np.nan)

In [9]:
# cram the dictionary into a dataframe
df = pd.DataFrame.from_dict(df_dict)

In [10]:
df.head(2)

,condominium,latitude,longitude,permittype,total_existing_bldg_sqft,total_job_valuation,total_new_add_sqft
0,NaN,30.22768623,-97.78692854,DS,NaN,NaN,NaN
1,No,30.16809048,-97.9131181,EP,NaN,NaN,1536


In [27]:
df.shape

(2000, 6)

# Let's page through the data

In [182]:
def page_api(n, api_key):
    
    # set endpoint URL
    url = 'https://data.austintexas.gov/resource/x9yh-78fz.json'
    # query params
    payload = { '$limit': 1500,
                '$order': ':id',
                '$offset': 1000,
                '$select': 'longitude, latitude, \
                    permittype, applieddate'}
    # API token
    headers = { 'X-App-Token': api_key }
    
    res = requests.get(url, params=payload, headers=headers)
    
    # create a template dictionary
    # consisting of all unique keys
    # within input dataset
    df_dict = {}
    for key in get_unique_keys(res.json()):
        df_dict[key] = []

    # fill in the dictionary with
    # values from the API call
    # making sure to input nulls
    # where needed
    for row in res.json():
        for k in list(df_dict.keys()):
            try:
                df_dict[k].append(row[k])
                continue
            except KeyError:
                df_dict[k].append(np.nan)
    
    df = pd.DataFrame.from_dict(df_dict)
    print('initial', df.shape)
    
    for offset, limit in get_page_indexes(n):
        payload['$limit'] = limit
        payload['$offset'] = offset
        print(payload)
        
        res = requests.get(url, params=payload, headers=headers)

        # fill in the dictionary with
        # values from the API call
        # making sure to input nulls
        # where needed
        for row in res.json():
            for k in list(df_dict.keys()):
                try:
                    df_dict[k].append(row[k])
                    continue
                except KeyError:
                    df_dict[k].append(np.nan)
    
        # cram the dictionary into a dataframe
        df_paged = pd.DataFrame.from_dict(df_dict)
        
        print(df_paged.shape)

        df_paged['applieddate'] = pd.to_datetime(df_paged.copy()['applieddate'], format='%Y-%m-%d', errors='coerce')

        permit_encoder = {'EP': 0, 'MP': 1, 'PP': 2, 'BP': 3, 'DS': 4}
        df_paged['permittype'] = df_paged.copy()['permittype'].apply(lambda x: permit_encoder[x] if x else 999)

        for i in ['latitude', 'longitude']:
            df_paged[i] = df_paged.copy()[i].astype(float)
            
        df.append(df_paged, ignore_index=True)
#         df = pd.concat([df, df_paged], axis=0, ignore_index=True)
            
    return df
    
        # send request to api endpoint
#         res = requests.get(url, params=payload, headers=headers)

In [183]:
page_api(1500, 'HLR3sDnaPUVVakzmZzKPpFMkJ')

initial (1500, 4)
{'$limit': 1000, '$order': ':id', '$offset': 1, '$select': 'longitude, latitude,                     permittype, applieddate'}
(2500, 4)
{'$limit': 1501, '$order': ':id', '$offset': 1001, '$select': 'longitude, latitude,                     permittype, applieddate'}
(4001, 4)


,applieddate,latitude,longitude,permittype
0,2007-03-13T00:00:00.000,30.31097355,-97.72413453,PP
1,2007-03-13T00:00:00.000,30.31097355,-97.72413453,EP
2,2007-03-13T00:00:00.000,30.31097355,-97.72413453,MP
3,2007-03-13T00:00:00.000,30.31097355,-97.72413453,PP
4,2007-03-15T00:00:00.000,30.18594826,-97.75125441,EP
5,2007-02-12T00:00:00.000,30.18093613,-97.91978591,BP
6,2007-02-12T00:00:00.000,30.18093613,-97.91978591,DS
7,2007-02-12T00:00:00.000,30.18093613,-97.91978591,EP
8,2007-02-12T00:00:00.000,30.18093613,-97.91978591,MP
9,2007-02-12T00:00:00.000,30.18093613,-97.91978591,PP


In [161]:
def get_page_indexes(n, page_size=1000):
    
    cur_row = 1
    page_size = page_size
    first_loop_flag = 1
    
    page_indexes = []
    
    while cur_row < n:
        if first_loop_flag == 1:
#             print('from: ', cur_row, ' offset: ', page_size )
            page_indexes.append([cur_row, page_size])
            cur_row += page_size
            first_loop_flag = 0
        else:
            if n < cur_row + page_size - 1:
#                 print('bang!')
#                 print('from: ', cur_row, ' offset: ', cur_row + n % page_size )
                page_indexes.append([cur_row, cur_row + n % page_size])
                cur_row += n % page_size
            else:
#                 print('from: ', cur_row, ' offset: ', cur_row + page_size - 1 )
                page_indexes.append([cur_row, cur_row + page_size - 1])
                cur_row += page_size
    
    return page_indexes

In [181]:
for i,j in get_page_indexes(1500):
    print(i,j)

1 1000
1001 1501


In [135]:
5050 % 1000

50

In [136]:
page_api(5050, 'foo')

from:  1  offset:  1000
from:  1001  offset:  2000
from:  2001  offset:  3000
from:  3001  offset:  4000
from:  4001  offset:  5000
bang!
from:  5001  offset:  5051


In [95]:
while cur_row < n:
    

5.5

In [66]:
'HLR3sDnaPUVVakzmZzKPpFMkJ'

In [68]:
res.status_code

200

In [69]:
# create a template dictionary
# consisting of all unique keys
# within input dataset
df_dict = {}
for key in get_unique_keys(res.json()):
    df_dict[key] = []
    
# fill in the dictionary with
# values from the API call
# making sure to input nulls
# where needed
for row in res.json():
    for k in list(df_dict.keys()):
        try:
            df_dict[k].append(row[k])
            continue
        except KeyError:
            df_dict[k].append(np.nan)

In [70]:
# cram the dictionary into a dataframe
df = pd.DataFrame.from_dict(df_dict)

df['applieddate'] = pd.to_datetime(df.copy()['applieddate'], format='%Y-%m-%d', errors='coerce')

permit_encoder = {'EP': 0, 'MP': 1, 'PP': 2, 'BP': 3, 'DS': 4}
df['permittype'] = df.copy()['permittype'].apply(lambda x: permit_encoder[x] if x else 999)

for i in ['latitude', 'longitude']:
    df[i] = df.copy()[i].astype(float)

In [93]:
df.head()

,applieddate,latitude,longitude,permittype,total_job_valuation
0,2007-04-02,30.154107,-97.788541,0,NaN
1,2007-04-02,30.154107,-97.788541,1,NaN
2,2007-04-02,30.154107,-97.788541,2,NaN
3,2006-09-29,30.260486,-97.732000,3,145000
4,2006-09-29,30.260486,-97.732000,4,NaN


In [94]:
df.dtypes

applieddate            datetime64[ns]
latitude                      float64
longitude                     float64
permittype                      int64
total_job_valuation            object
dtype: object

In [86]:
df['permittype'].unique()

array([0, 1, 2, 3, 4])

In [60]:
pd.concat([df, pd.DataFrame.from_dict(df_dict)], axis=0, ignore_index=True)

,latitude,longitude,permittype,total_existing_bldg_sqft,total_job_valuation,total_new_add_sqft
0,30.25382724,-97.71579756,BP,NaN,90307,849
1,30.18917783,-97.75088001,BP,NaN,NaN,NaN
2,30.26009874,-97.72824677,EP,NaN,NaN,NaN
3,30.28191734,-97.66106437,EP,NaN,NaN,NaN
4,30.28191734,-97.66106437,MP,NaN,NaN,NaN
5,30.28191734,-97.66106437,PP,NaN,NaN,NaN
6,30.20560396,-97.83157149,BP,NaN,NaN,NaN
7,30.30284798,-97.66556269,BP,NaN,NaN,NaN
8,30.24055526,-97.74986085,BP,NaN,82000,1648
9,30.24055526,-97.74986085,EP,1452,NaN,1648


In [54]:
pd.DataFrame.from_dict(df_dict).shape

(2000, 6)

In [42]:
df.shape

(2000, 6)

In [ ]:
def page_api(n, api_key):
    
    # set endpoint URL
    url = 'https://data.austintexas.gov/resource/x9yh-78fz.json'
    # query params
    payload = { '$limit': 2000,
                '$order': ':id',
                '$offset': 4000,
                '$select': 'longitude, latitude, \
                    permittype, applieddate'}
    # API token
    headers = { 'X-App-Token': api_key }
    
    for offset, limit in get_page_indexes(n):
        payload['$limit'] = limit
        payload['$offset'] = offset
        print(payload)
        
        res = requests.get(url, params=payload, headers=headers)
        
        # create a template dictionary
        # consisting of all unique keys
        # within input dataset
        df_dict = {}
        for key in get_unique_keys(res.json()):
            df_dict[key] = []

        # fill in the dictionary with
        # values from the API call
        # making sure to input nulls
        # where needed
        for row in res.json():
            for k in list(df_dict.keys()):
                try:
                    df_dict[k].append(row[k])
                    continue
                except KeyError:
                    df_dict[k].append(np.nan)
    
        # cram the dictionary into a dataframe
        df = pd.DataFrame.from_dict(df_dict)
        
        print(df.shape)

        df['applieddate'] = pd.to_datetime(df.copy()['applieddate'], format='%Y-%m-%d', errors='coerce')

        permit_encoder = {'EP': 0, 'MP': 1, 'PP': 2, 'BP': 3, 'DS': 4}
        df['permittype'] = df.copy()['permittype'].apply(lambda x: permit_encoder[x] if x else 999)

        for i in ['latitude', 'longitude']:
            df[i] = df.copy()[i].astype(float)
            
    return df
    
        # send request to api endpoint
#         res = requests.get(url, params=payload, headers=headers)